In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

## Parameters

In [2]:
class Config:
    # Embeddings
    embedding_dim = 300
    
    # RNN
    hidden_size = 300
    
    # Training parameters
    batch_size = 64
    num_epochs = 20
    display_every = 500
    evaluate_every = 1000
    num_checkpoints = 5
    learning_rate = 0.001
    decay_rate = 0.9
    
    # Testing parameters
    checkpoint_dir = ''
    
    UNK = "$UNK$"
    NUM = "$NUM$"
    NONE = "O"
    PAD = '$PAD$'

## Dataset 

Load annotated corpus for named entity recognition

https://www.kaggle.com/abhinavwalia95/entity-annotated-corpus

In [3]:
import nltk
import re
import os

class Dataset:
    def __init__(self):
        self.all_tags, self.all_words = [], [] 
        
    def processing_word(self, word):
        word = word.lower()
        if word.isdigit():
            word = Config.NUM
        return word
        
    def load_dataset(self, path):
        words_col, tags_col = [], []
        with open(path) as f:
            words, tags = [], []
            for line in f:
                line = line.strip()
                if (len(line) == 0 or line.startswith("-DOCSTART-")):
                    if len(words) != 0:
                        words_col.append(' '.join(words))
                        tags_col.append(' '.join(tags))
                        words, tags = [], []
                else:
                    ls = line.split(' ')
                    word, tag = ls[0],ls[3]
                    word = self.processing_word(word)
                    
                    words.append(word)
                    tags.append(tag)
                    
                    self.all_words.append(word)
                    self.all_tags.append(tag)
                    
        return pd.DataFrame({'words': words_col, 'tags': tags_col})
        
    def download_and_load_datasets(self):
        self.all_tags, self.all_words = [], [] 
        
        dataset = tf.keras.utils.get_file(
          fname="CoNLL-2003.zip", 
          origin="https://s3.ap-northeast-2.amazonaws.com/bowbowbow-storage/dataset/CoNLL-2003.zip", 
          extract=True)
        
        dir_path = os.path.join(os.path.dirname(dataset), 'CoNLL-2003')
        train_df = self.load_dataset(os.path.join(dir_path, 'eng.train'))
        dev_df = self.load_dataset(os.path.join(dir_path, 'eng.testa'))
        test_df = self.load_dataset(os.path.join(dir_path, 'eng.testb'))
        return train_df, dev_df, test_df

dataset = Dataset()
train_df, dev_df, test_df = dataset.download_and_load_datasets()
train_df.head(10)

,tags,words
0,I-ORG O I-MISC O O O I-MISC O O,eu rejects german call to boycott british lamb .
1,I-PER I-PER,peter blackburn
2,I-LOC O,brussels 1996-08-22
3,O I-ORG I-ORG O O O O O O I-MISC O O O O O I-M...,the european commission said on thursday it di...
4,I-LOC O O O O I-ORG I-ORG O O O I-PER I-PER O ...,germany 's representative to the european unio...
5,O O O O O O O O O O O O O O O O O O O O I-ORG ...,""" we do n't support any such recommendation be..."
6,O O O O O O O O O O O O O O O O O O O O O O I-...,he said further scientific study was required ...
7,O O O O O O O I-ORG O O I-PER I-PER O O O O O ...,he said a proposal last month by eu farm commi...
8,I-PER O I-MISC O O O O I-LOC O I-LOC O O O O O...,fischler proposed eu-wide measures after repor...
9,O I-PER O O O O O O O I-ORG O O O O O O O O O ...,but fischler agreed to review his proposal aft...


In [4]:
word_list = list(set(dataset.all_words)) + [Config.PAD, Config.UNK]
word2idx = {w: i for i, w in enumerate(word_list)}
idx2word = {i: w for i, w in enumerate(word_list)}

tag_list = list(set(dataset.all_tags))
tag2idx = {w: i for i, w in enumerate(tag_list)}
idx2tag = {i: w for i, w in enumerate(tag_list)}

## Model

In [5]:
class Model:
    def __init__(self, 
               num_classes, 
               vocab_size, 
               embedding_size, 
               hidden_size):
        
        self.word_ids = tf.placeholder(tf.int32, shape=[None, None], name='word_ids')
        self.labels = tf.placeholder(tf.int32, shape=[None, None], name='labels')
        self.sequence_lengths = tf.placeholder(tf.int32, shape=[None], name="sequence_lengths")
        self.dropout = tf.placeholder(dtype=tf.float32, shape=[],name="dropout")
        
        initializer = tf.contrib.layers.xavier_initializer()
        
        # Embedding layer
        with tf.variable_scope('text-embedding'):
            self.W_text = tf.Variable(tf.random_uniform([vocab_size, embedding_size], -0.25, 0.25), name='W_text', trainable=False)
            self.embedded_chars = tf.nn.embedding_lookup(self.W_text, self.word_ids) # [batch_size, max_sequence_length, embedding_size]
            
        # Bidirectional LSTM
        with tf.variable_scope("bi-lstm"):
            fw_cell = tf.nn.rnn_cell.LSTMCell(hidden_size)
            bw_cell = tf.nn.rnn_cell.LSTMCell(hidden_size)
            (output_fw, output_bw), _ = tf.nn.bidirectional_dynamic_rnn(cell_fw=fw_cell,
                                                                  cell_bw=bw_cell,
                                                                  inputs=self.embedded_chars,
                                                                  sequence_length= self.sequence_lengths, # [batch_size],
                                                                  dtype=tf.float32)
            
            self.rnn_outputs = tf.concat([output_fw, output_bw], axis=-1)  # [batch_size, max_sequence_length, 2*hidden_size]
            self.rnn_outputs = tf.nn.dropout(self.rnn_outputs, self.dropout)
    
    
        # Fully connected layer
        with tf.variable_scope('output'):
            self.W_output = tf.get_variable('W_output', shape=[2*hidden_size, num_classes],  dtype=tf.float32)
            self.b_output = tf.get_variable('b_output', shape=[num_classes], dtype=tf.float32, initializer=tf.zeros_initializer())
            
            nsteps = tf.shape(self.rnn_outputs)[1]
            rnn_outputs_flat = tf.reshape(self.rnn_outputs, [-1, 2*hidden_size])
            pred = tf.matmul(rnn_outputs_flat, self.W_output) + self.b_output
            
            self.logits = tf.reshape(pred, [-1, nsteps, num_classes]) # [batch_size, max_sequence_length, num_classes]
    
        # Calculate mean corss-entropy loss
        with tf.variable_scope('loss'):
            self.losses = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=self.logits, labels=self.labels)
            mask = tf.sequence_mask(self.sequence_lengths)
            losses = tf.boolean_mask(self.losses, mask)
            
            self.loss = tf.reduce_mean(losses) 
        
        # Accuracy    
        with tf.name_scope('accuracy'):
            self.predictions = tf.argmax(self.logits, 2, name='predictions')
            correct_predictions = tf.equal(self.predictions, tf.cast(self.labels, tf.int64))
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32), name='accuracy')
    
    # Length of the sequence data
    @staticmethod
    def _length(seq):
        relevant = tf.sign(tf.abs(seq))
        length = tf.reduce_sum(relevant, reduction_indices=1)
        length = tf.cast(length, tf.int32)
        return length

## Function for train

In [6]:
# Pre-trained glove
def load_glove(embedding_dim, word2idx):
    download_path = tf.keras.utils.get_file(
      fname="glove.6B.zip", 
      origin="http://nlp.stanford.edu/data/glove.6B.zip", 
      extract=True)
    
    embedding_path = os.path.join(os.path.dirname(download_path), 'glove.6B.300d.txt')
    print('embedding_path :', embedding_path)

    # initial matrix with random uniform
    initW = np.random.randn(len(word2idx), embedding_dim).astype(np.float32) / np.sqrt(len(word2idx))
    # load any vectors from the glove
    print("Load glove file {0}".format(embedding_path))
    f = open(embedding_path, 'r', encoding='utf8')
    for line in f:
        splitLine = line.split(' ')
        word = splitLine[0]
        embedding = np.asarray(splitLine[1:], dtype='float32')
        if word in word2idx:
            initW[word2idx[word]] = embedding
    return initW

def batch_iter(df, batch_size, num_epochs, shuffle=True, tqdm_disable=False):
    """
    Generates a batch iterator for a dataset.
    """
    data_size = len(df)
    num_batches_per_epoch = int((data_size - 1) / batch_size) + 1
    for epoch in tqdm(range(num_epochs), disable=tqdm_disable):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_df= df.iloc[shuffle_indices]
        else:
            shuffled_df = df
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_df.iloc[start_index:end_index]    
            

def get_feed_dict(batch_df):
    max_length = max(map(lambda x : len(x), batch_df['words'].tolist()))

    word_ids, sequence_lengths, labels = [], [], []
    for index, row in batch_df.iterrows():
        sentence = row['words'].split(' ')
        tags = row['tags'].split(' ')

        word_ids_row, labels_row = [], []
        for word in sentence:
            word_ids_row.append(word2idx[word])
        for tag in tags:
            labels_row.append(tag2idx[tag])

        if len(sentence) < max_length:
            word_ids_row += [word2idx[Config.PAD]]* (max_length - len(sentence))
            labels_row += [tag2idx[Config.NONE]]* (max_length - len(sentence))

        word_ids.append(word_ids_row)
        labels.append(labels_row)
        sequence_lengths.append(len(sentence))

    word_ids = np.array(word_ids)
    labels = np.array(labels)
    sequence_lengths = np.array(sequence_lengths)
    return word_ids, labels, sequence_lengths

def evaluation(y, preds, lengths):
    from sklearn.metrics import classification_report
    arg_answer, arg_pred = [], []
    for i in range(len(y)):
        for j in range(lengths[i]):
            arg_answer.append(idx2tag[y[i][j]])
            arg_pred.append(idx2tag[preds[i][j]])

    print(classification_report(arg_answer, arg_pred))

## Train

In [7]:
import datetime
import time

from sklearn.metrics import f1_score
import warnings
import sklearn.exceptions
warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)

tf.reset_default_graph()
sess = tf.Session()
with sess.as_default():
    model = Model(
        num_classes=len(tag_list),
        vocab_size=len(word_list),
        embedding_size=Config.embedding_dim,
        hidden_size=Config.hidden_size
    )
    
    global_step = tf.Variable(0, name='global_step', trainable=False)
    train_op = tf.train.AdamOptimizer(Config.learning_rate).minimize(model.loss, global_step=global_step)
    
    # Output directory for models and summary
    timestamp = str(int(time.time()))
    out_dir = os.path.abspath(os.path.join(os.path.curdir, "30.runs", timestamp))
    print("Writing to {}\n".format(out_dir))

    sess.run(tf.global_variables_initializer())

    pretrain_W = load_glove(Config.embedding_dim, word2idx)
    sess.run(model.W_text.assign(pretrain_W))
    print("Success to load pre-trained glove model!\n")
    
    # Generate batches
    batches = batch_iter(train_df, Config.batch_size, Config.num_epochs)
    for batch_df in batches:
        word_ids, labels, sequence_lengths = get_feed_dict(batch_df)
        feed_dict = {
            model.word_ids: word_ids,
            model.labels: labels,
            model.sequence_lengths: sequence_lengths,
            model.dropout: 0.5,
        }
        _, step, loss, accuracy, predictions = sess.run([
            train_op, global_step, model.loss, model.accuracy, model.predictions], feed_dict)

        # Training log display
        if step % Config.display_every == 0:
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            
        # Evaluation
        if step % Config.evaluate_every == 0:
            batches = batch_iter(dev_df, Config.batch_size, 1, tqdm_disable=True)
            
            total_loss, predictions_all, labels_all, sequence_lengths_all  = 0, [], [], []
            for batch_df in batches:
                word_ids, labels, sequence_lengths = get_feed_dict(batch_df)
                feed_dict = {
                    model.word_ids: word_ids,
                    model.labels: labels,
                    model.sequence_lengths: sequence_lengths,
                    model.dropout: 1.0,
                }
                loss, predictions = sess.run([model.loss, model.predictions], feed_dict)
                total_loss += loss
                predictions_all += predictions.tolist()
                labels_all += labels.tolist()
                sequence_lengths_all += sequence_lengths.tolist()
        
            time_str = datetime.datetime.now().isoformat()
            print("\nDev Evaluation\n{}: loss {:g}\n".format(time_str, total_loss/len(predictions_all)))
            evaluation(labels_all, predictions_all, sequence_lengths_all)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/home/seungwon/project/tf-notes/venv/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Writing to /home/seungwon/project/tf-notes/30.runs/1558721235

embedding_path : /home/seungwon/.keras/datasets/glove.6B.300d.txt
Load glove file /home/seungwon/.keras/datasets/glove.6B.300d.txt


  0%|          | 0/20 [00:00<?, ?it/s]

Success to load pre-trained glove model!



...

 75%|███████▌  | 15/20 [03:32<01:11, 14.31s/it]

2019-05-25T03:11:39.492185: step 3500, loss 0.0865599, acc 0.998922


 90%|█████████ | 18/20 [04:13<00:27, 13.92s/it]

2019-05-25T03:12:10.659818: step 4000, loss 0.0610869, acc 0.998503

Dev Evaluation
2019-05-25T03:12:12.618599: loss 0.00102861

              precision    recall  f1-score   support

      B-MISC       0.00      0.00      0.00         4
       I-LOC       0.95      0.90      0.93      2094
      I-MISC       0.90      0.74      0.81      1264
       I-ORG       0.85      0.79      0.82      2092
       I-PER       0.97      0.95      0.96      3149
           O       0.98      0.99      0.99     42759

   micro avg       0.97      0.97      0.97     51362
   macro avg       0.77      0.73      0.75     51362
weighted avg       0.97      0.97      0.97     51362



100%|██████████| 20/20 [04:43<00:00, 14.24s/it]


In [8]:
batches = batch_iter(test_df, Config.batch_size, 1)
total_loss, predictions_all, labels_all, sequence_lengths_all  = 0, [], [], []
for batch_df in batches:
    word_ids, labels, sequence_lengths = get_feed_dict(batch_df)
    feed_dict = {
        model.word_ids: word_ids,
        model.labels: labels,
        model.sequence_lengths: sequence_lengths,
        model.dropout: 1.0,
    }
    
    total_loss, predictions = sess.run([model.loss, model.predictions], feed_dict)
    total_loss += loss
    predictions_all += predictions.tolist()
    labels_all += labels.tolist()
    sequence_lengths_all += sequence_lengths.tolist()

time_str = datetime.datetime.now().isoformat()
print("\nTest Evaluation\n{}: loss {:g}\n".format(time_str, total_loss/len(predictions_all)))
evaluation(labels_all, predictions_all, sequence_lengths_all)

100%|██████████| 1/1 [00:02<00:00,  2.02s/it]


Test Evaluation
2019-05-25T03:12:39.682714: loss 2.80892e-05

              precision    recall  f1-score   support

       B-LOC       0.00      0.00      0.00         6
      B-MISC       0.00      0.00      0.00         9
       B-ORG       0.00      0.00      0.00         5
       I-LOC       0.88      0.90      0.89      1919
      I-MISC       0.77      0.68      0.72       909
       I-ORG       0.81      0.81      0.81      2491
       I-PER       0.97      0.92      0.95      2773
           O       0.98      0.99      0.98     38323

   micro avg       0.96      0.96      0.96     46435
   macro avg       0.55      0.54      0.54     46435
weighted avg       0.96      0.96      0.96     46435



## Tensorboard

```
tensorboard --logdir=./30.runs --host 0.0.0.0
```